In [1]:
library(geojsonio)
library(tidyverse)
library(dplyr)
library("knitr")

Registered S3 method overwritten by 'geojsonsf':
  method        from   
  print.geojson geojson


Attaching package: 'geojsonio'


The following object is masked from 'package:base':

    pretty


-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v dplyr     1.1.4     v readr     2.1.5
v forcats   1.0.0     v stringr   1.5.0
v ggplot2   3.5.1     v tibble    3.2.1
v lubridate 1.9.3     v tidyr     1.3.1
v purrr     1.0.2     
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:

library(cancensus)
options(cancensus.api_key='CensusMapper_f228791d9506a7a747ece66db73b95be')
options(cancensus.cache_path = "/cache")

Census data is currently stored temporarily.

 In order to speed up performance, reduce API quota usage, and reduce unnecessary network calls, please set up a persistent cache directory via `set_cancensus_cache_path('<local cache path>', install = TRUE)`.
 This will add your cache directory as environment varianble to your .Renviron to be used across sessions and projects.





In [3]:
census_data <- get_census(dataset='CA16', regions=list(CSD="5915022"), vectors=c("v_CA16_401","v_CA16_404","v_CA16_406","v_CA16_1","v_CA16_2396","v_CA16_2510"), labels="detailed", geo_format="sf", level='DA')

Reading vectors data from local cache.

Reading geo data from local cache.



In [5]:
library("sf")
library(VancouvR)
search_cov_datasets("property-tax") %>%
  select(dataset_id,title)

dataset_id,title
<chr>,<chr>
property-tax-report-2006-2010,Property tax report 2006-2010
property-tax-report-2011-2015,Property tax report 2011-2015
property-tax-report-2016-2019,Property tax report 2016-2019
property-tax-report,Property tax report


In [6]:
tax_data <- get_cov_data(dataset_id = "property-tax-report-2016-2019",
                         where="tax_assessment_year='2016'",
                         select = "current_land_value, land_coordinate as tax_coord")|>
    as.tibble()
glimpse(tax_data)

Warning message:
"`as.tibble()` was deprecated in tibble 2.0.0.
i Please use `as_tibble()` instead.
i The signature and semantics have changed, see `?as_tibble`."

Warning message:
"There was 1 warning in `mutate()`.
i In argument: `current_land_value =
  .Primitive("as.integer")(current_land_value)`.
Caused by warning:
! NAs introduced by coercion to integer range"


Rows: 199,924
Columns: 2
$ current_land_value <int> 126000, 1100000, 1028000, 825000, 922000, 878000, 1~
$ tax_coord          <chr> "58825549", "60125529", "61223603", "63025324", "59~


In [7]:
property_polygons <- get_cov_data(dataset_id="property-parcel-polygons")

In [8]:
land_price <- merge(tax_data, property_polygons, by = "tax_coord")|>
    glimpse() 
land_price_sf <- st_sf(land_price, geometry = land_price$geometry, crs = 4326)

Rows: 204,278
Columns: 8
$ tax_coord          <chr> "01963206", "01963229", "01963245", "01963265", "01~
$ current_land_value <int> 11366000, 10623000, 10763000, 10763000, 12864000, 1~
$ civic_number       <chr> "4888", "4898", "4874", "4850", "4811", "4818", "16~
$ streetname         <chr> "BELMONT AV", "FANNIN AV", "FANNIN AV", "FANNIN AV"~
$ site_id            <chr> "010862706", "010863915", "010863923", "010863931",~
$ geom               <chr> "{\"coordinates\": [[[-123.22221178283283, 49.27466~
$ geo_point_2d       <chr> "49.27488185499064, -123.22286857659084", "49.27431~
$ geometry           <POLYGON [°]> POLYGON ((-123.2222 49.2746..., POLYGON ((-~


In [ ]:
library(ggmap)

In [9]:
crime_data <- read_csv('crime_data_kraggle/crime.csv')

Rows: 530652 Columns: 12
-- Column specification --------------------------------------------------------
Delimiter: ","
chr (3): TYPE, HUNDRED_BLOCK, NEIGHBOURHOOD
dbl (9): YEAR, MONTH, DAY, HOUR, MINUTE, X, Y, Latitude, Longitude

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [10]:
crime_data_sf <- st_as_sf(crime_data, coords = c("Longitude", "Latitude"), crs = 4326)

In [ ]:
glimpse(crime_data_sf)

Rows: 530,652
Columns: 12
$ TYPE          <chr> "Other Theft", "Other Theft", "Other Theft", "Other Thef~
$ YEAR          <dbl> 2003, 2003, 2003, 2003, 2003, 2003, 2003, 2003, 2003, 20~
$ MONTH         <dbl> 5, 5, 4, 4, 4, 3, 3, 6, 2, 7, 1, 9, 4, 9, 11, 9, 10, 1, ~
$ DAY           <dbl> 12, 7, 23, 20, 12, 26, 10, 28, 16, 9, 31, 27, 19, 24, 5,~
$ HOUR          <dbl> 16, 15, 16, 11, 17, 20, 12, 4, 9, 18, 19, 1, 18, 18, 8, ~
$ MINUTE        <dbl> 15, 20, 40, 15, 45, 45, 0, 13, 2, 15, 45, 0, 0, 30, 12, ~
$ HUNDRED_BLOCK <chr> "9XX TERMINAL AVE", "9XX TERMINAL AVE", "9XX TERMINAL AV~
$ NEIGHBOURHOOD <chr> "Strathcona", "Strathcona", "Strathcona", "Strathcona", ~
$ X             <dbl> 493906.5, 493906.5, 493906.5, 493906.5, 493906.5, 493906~
$ Y             <dbl> 5457452, 5457452, 5457452, 5457452, 5457452, 5457452, 54~
$ Latitude      <dbl> 49.26980, 49.26980, 49.26980, 49.26980, 49.26980, 49.269~
$ Longitude     <dbl> -123.0838, -123.0838, -123.0838, -123.0838, -123.0838, -~


In [ ]:
polygons_sf <- st_as_sf(polygons_df, wkt = "geometry", crs = 4326)